In [15]:
# [1] imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [16]:
# [2] Create Convolutional neural network (Better for Images)
class CNN(nn.Module):
    # in_channels: 1 black, 3 rgb
    def __init__(self, in_channels = 1, num_classes = 10):  
        super(CNN, self).__init__()
        # same convolution, output size will change, but wwe keep same dimension
        # n_out = [(n_in + 2padding - kernel_size)/stride]+1 = [(28 + 2 * 1 - 3)/1]+1 = 27 + 1 = 28
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) 
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride = (2,2)) #to14
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1)) 
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x
        
    
'''
# Basic model Checks
model = CNN()
x = torch.rand(64, 1, 28, 28)
print(model(x).shape)
'''


'\n# Basic model Checks\nmodel = CNN()\nx = torch.rand(64, 1, 28, 28)\nprint(model(x).shape)\n'

In [17]:
# [3] Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(device)

In [18]:
# [4] Hyperparameters
INPUT_SIZE = 1
NUM_CLASSES = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 8

In [19]:
# [5] Load data
# Training dataset and loader
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# Testing dataset and loader
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
# [6] Initialize network
model = CNN().to(device)

In [21]:
# [7] Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [22]:
# [8] Train Network
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss =  criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()

In [23]:
# [9] Check Accuracy on training and test to see how good is my model
def chackAccuracy(loader, model):
        if loader.dataset.train:
            print("Checking Accuracy on TRAIN data")
        else:
            print("Checking Accuracy on TEST data")
        num_correct = 0
        num_samples = 0
        model.eval()

        with torch.no_grad():
            for x,y in loader:
                x = x.to(device)
                y = y.to(device)

                scores = model(x)
                _, predictions = scores.max(1)
                num_correct += (predictions == y).sum()
                num_samples += predictions.size(0)
            print(f'Got {num_correct} / {num_samples} wich accuracy {float(num_correct)/float(num_samples)*100:.2f}')

        model.train()

chackAccuracy(train_loader, model)
chackAccuracy(test_loader, model)

Checking Accuracy on TRAIN data
Got 59328 / 60000 wich accuracy 98.88
Checking Accuracy on TEST data
Got 9859 / 10000 wich accuracy 98.59
